In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import os
from dotenv import load_dotenv

load_dotenv()
# environment variables

True

In [2]:
# Initialize openai API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### 0. Simple Input/Output Still works
You just need to pass it as a message instead. More on this in a second.

In [3]:
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [4]:
message = [HumanMessage(content="What is the name of the most populous state in the USA?")]

chat(message)

AIMessage(content='California.', additional_kwargs={})

### 1. Chat Messages
* HumanMessage: A message sent from the perspective of the human
* AIMessage: A message sent from the perspective of the AI the human is interacting with
* SystemMessage: A message setting the objectives the AI should follow
* ChatMessage: A message allowing for arbitrary setting of role. You won’t be using this too much

In [5]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(content='I hate programming.', additional_kwargs={})

In [6]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming."),
    AIMessage(content='I hate programming.'),
    HumanMessage(content="The moon is out")
]
chat(messages)

AIMessage(content='The moon is not out.', additional_kwargs={})

In [7]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming."),
    AIMessage(content='I hate programming.'),
    HumanMessage(content="What is the first thing that I said?")
]
chat(messages)

AIMessage(content='You said, "I love programming."', additional_kwargs={})

### Batch Messages

In [8]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful word machine that creates an alliteration using a base word"),
        HumanMessage(content="Base word: Apple")
    ],
    [
        SystemMessage(content="You are a helpful word machine that creates an alliteration using a base word"),
        HumanMessage(content="Base word: Dog")
    ],
]
chat.generate(batch_messages)

LLMResult(generations=[[ChatGeneration(text='Apples are an awesome autumnal accessory.', generation_info=None, message=AIMessage(content='Apples are an awesome autumnal accessory.', additional_kwargs={}))], [ChatGeneration(text='Delightful dogs dig deep in dirt.', generation_info=None, message=AIMessage(content='Delightful dogs dig deep in dirt.', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 64, 'completion_tokens': 18, 'total_tokens': 82}, 'model_name': 'gpt-3.5-turbo'})

### Prompt Templates
With one or more `MessagePromptTemplates` you can build a `ChatPromptTemplate`

In [9]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template="Propose creative ways to incorporate {food_1} and {food_2} in the cuisine of the users choice.",
    input_variables=["food_1", "food_2"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [10]:
# Output of system_message_prompt
system_message_prompt.format(food_1="Bacon", food_2="Shrimp")

SystemMessage(content='Propose creative ways to incorporate Bacon and Shrimp in the cuisine of the users choice.', additional_kwargs={})

In [11]:
# Make HumanMessagePromptTemplate
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [12]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['text', 'food_1', 'food_2'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['food_1', 'food_2'], output_parser=None, partial_variables={}, template='Propose creative ways to incorporate {food_1} and {food_2} in the cuisine of the users choice.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='{text}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [13]:
chat_prompt_with_values = chat_prompt.format_prompt(food_1="Bacon", \
                                                   food_2="Shrimp", \
                                                   text="I really like food from Germany.")

chat_prompt_with_values.to_messages()

[SystemMessage(content='Propose creative ways to incorporate Bacon and Shrimp in the cuisine of the users choice.', additional_kwargs={}),
 HumanMessage(content='I really like food from Germany.', additional_kwargs={})]

In [14]:
response = chat(chat_prompt_with_values.to_messages()).content
print (response)

Great choice! Here are some creative ways to incorporate bacon and shrimp into German cuisine:

1. Bacon and shrimp spätzle: Spätzle is a traditional German egg noodle dish. Add diced bacon and sautéed shrimp to the spätzle dough, then cook and serve with a cream sauce.

2. Shrimp and bacon sauerkraut balls: Mix cooked shrimp and crumbled bacon into a mixture of sauerkraut, bread crumbs, and seasonings. Form into balls and fry until golden brown.

3. Bacon-wrapped shrimp skewers: Wrap cooked shrimp in thin slices of bacon and grill or broil until the bacon is crispy. Serve with a side of German potato salad.

4. Shrimp and bacon strudel: Roll cooked shrimp and bacon into layers of phyllo dough with sautéed onions, spinach, and cream cheese. Bake until golden brown and serve as a main dish or appetizer.

5. Bacon and shrimp bratwurst: Mix diced shrimp and bacon into ground pork along with traditional bratwurst seasonings. Form into sausage links and grill or pan-fry until cooked through

### With Streaming

In [15]:
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0)

resp = chat(chat_prompt_with_values.to_messages())

Great choice! Here are some creative ways to incorporate bacon and shrimp into German cuisine:

1. Bacon and Shrimp Spätzle: Spätzle is a traditional German egg noodle dish. Add cooked shrimp and crispy bacon to the spätzle and toss with butter and herbs for a delicious and hearty meal.

2. Shrimp and Bacon Sauerkraut: Add cooked shrimp and crispy bacon to sauerkraut for a flavorful and protein-packed side dish. Serve alongside traditional German sausages or pork dishes.

3. Bacon and Shrimp Potato Salad: Add cooked shrimp and crispy bacon to a classic German potato salad for a unique twist on a traditional dish. The salty bacon and sweet shrimp will add a delicious flavor to the creamy potato salad.

4. Shrimp and Bacon Schnitzel: Top a crispy schnitzel with cooked shrimp and crispy bacon for a decadent and flavorful meal. Serve with a side of German potato salad or sauerkraut.

5. Bacon and Shrimp Bratwurst: Mix cooked shrimp and crispy bacon into the bratwurst mixture before grillin